In [6]:
import pandas as pd

# 1. CSV 파일 불러오기
file_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\버스전용차로\기흥IC~신갈JC_차로변경_10%_2km_100-50_006.csv"
df = pd.read_csv(file_path, low_memory=False)

# 2. 필요한 컬럼만 추출 (메모리 최적화)
df = df[["NO", "VEHTYPE", "LANE\\LINK\\NO", "LANE\\INDEX", "$VEHICLE:SIMSEC", "POS", "SPEED"]]

# 3. A 차량 필터링: 링크번호 18번, 5차로, 버스
a_df = df[(df["LANE\\LINK\\NO"] == 18) & (df["LANE\\INDEX"] == 5) & (df["VEHTYPE"] == 300)].copy()

# 4. B 차량 후보: 4차로만 필터링 + 시간 기준으로 그룹화
df_4th_lane = df[df["LANE\\INDEX"] == 4].copy()
df_4th_lane_grouped = df_4th_lane.groupby("$VEHICLE:SIMSEC")
display(df_4th_lane_grouped)

# 5. 결과 저장 리스트
results = []

# 6. 샘플로 A 차량 일부만 선택 (전체 적용 시 a_df로 변경 가능)
a_sample = a_df

# 7. A 차량 기준으로 가장 가까운 B + B의 전방 C 추출
for _, a in a_sample.iterrows():
    simsec = a["$VEHICLE:SIMSEC"]
    a_pos = a["POS"]

    if simsec not in df_4th_lane_grouped.groups:
        continue

    b_set = df_4th_lane_grouped.get_group(simsec)
    if b_set.empty:
        continue

    # 가장 가까운 위치의 B 차량
    b_row = b_set.loc[(b_set["POS"] - a_pos).abs().idxmin()]

    # B 차량의 전방 C 차량
    front_candidates = b_set[b_set["POS"] > b_row["POS"]]
    if not front_candidates.empty:
        c_row = front_candidates.loc[front_candidates["POS"].idxmin()]
        results.append({
            "A_NO": a["NO"], "A_POS": a["POS"], "A_SPEED": a["SPEED"],
            "B_NO": b_row["NO"], "B_POS": b_row["POS"], "B_SPEED": b_row["SPEED"],
            "C_NO": c_row["NO"], "C_POS": c_row["POS"], "C_SPEED": c_row["SPEED"],
            "SIMSEC": simsec
        })
    else:
        results.append({
            "A_NO": a["NO"], "A_POS": a["POS"], "A_SPEED": a["SPEED"],
            "B_NO": b_row["NO"], "B_POS": b_row["POS"], "B_SPEED": b_row["SPEED"],
            "C_NO": None, "C_POS": None, "C_SPEED": None,
            "SIMSEC": simsec
        })

# 8. 결과 DataFrame 생성
result_df = pd.DataFrame(results)

display(result_df)
# 결과 확인 (예: result_df.head())


excel_file_path = r"C:\Users\(주)내일이엔시 도로교통안전연구소\Desktop\버스전용차로\result.xlsx"
#result_df.to_excel(excel_file_path, index=True)


,A_NO,A_POS,A_SPEED,B_NO,B_POS,B_SPEED,C_NO,C_POS,C_SPEED,SIMSEC
0,1426.0,159.171,101.64,1414.0,171.107,51.69,1409.0,252.235,49.36,901.0
1,1410.0,485.052,99.90,1384.0,477.854,51.52,1377.0,501.063,49.01,901.0
2,1393.0,832.633,97.37,1337.0,822.766,50.57,1336.0,884.287,52.98,901.0
3,1385.0,863.877,93.84,1336.0,884.287,52.98,1333.0,903.984,51.47,901.0
4,1382.0,900.715,88.12,1333.0,903.984,51.47,1329.0,931.150,47.50,901.0
...,...,...,...,...,...,...,...,...,...,...
49116,2943.0,5967.881,100.00,2594.0,5850.610,49.98,2580.0,6104.931,48.58,2080.0
49117,2928.0,6209.404,100.00,2543.0,6218.814,51.32,2539.0,6240.040,51.00,2080.0
49118,2903.0,6533.201,100.00,2521.0,6538.070,46.50,2517.0,6558.275,46.57,2080.0
49119,2896.0,6640.993,100.00,2533.0,6642.472,49.89,2489.0,6659.384,48.37,2080.0
